<a href="https://colab.research.google.com/github/mrm8488/shared_colab_notebooks/blob/master/HF_SetFit_multi_amazon_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SetFit for Multilingual Text Classification (binary)

In [ ]:
! nvidia-smi

Mon Oct 17 16:14:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   28C    P0    44W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
! pip install -q transformers datasets setfit

     |████████████████████████████████| 5.3 MB 4.6 MB/s 
     |████████████████████████████████| 441 kB 80.5 MB/s 
     |████████████████████████████████| 7.6 MB 69.5 MB/s 
     |████████████████████████████████| 163 kB 87.9 MB/s 
     |████████████████████████████████| 212 kB 90.0 MB/s 
     |████████████████████████████████| 115 kB 88.4 MB/s 
     |████████████████████████████████| 127 kB 93.0 MB/s 
     |████████████████████████████████| 85 kB 6.5 MB/s 
     |████████████████████████████████| 362 kB 69.3 MB/s 
     |████████████████████████████████| 69 kB 10.5 MB/s 
     |████████████████████████████████| 1.3 MB 74.2 MB/s 


In [ ]:
! huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens .
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your termin

In [ ]:
! git config --global credential.helper store

In [ ]:
from datasets import load_dataset
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitModel, SetFitTrainer

In [ ]:
dataset = load_dataset("amazon_reviews_multi")

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 1200000
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 30000
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 30000
    })
})

In [ ]:
from collections import Counter
counts = Counter(dataset['train']['stars'])
print(counts)

Counter({1: 240000, 2: 240000, 3: 240000, 4: 240000, 5: 240000})


In [ ]:
dataset = dataset.filter(lambda example: example["stars"] != 3)

In [ ]:
dataset = dataset.rename_column('review_body', 'text')

In [ ]:
def format_ds(example):
  example['label'] = 0 if example['stars'] < 3 else 1
  return example

In [ ]:
dataset = dataset.map(format_ds, batched=False, remove_columns=['review_id', 'product_id', 'reviewer_id', 'stars', 'review_title', 'language', 'product_category'])

  0%|          | 0/960000 [00:00<?, ?ex/s]

  0%|          | 0/24000 [00:00<?, ?ex/s]

  0%|          | 0/24000 [00:00<?, ?ex/s]

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
dataset['train']

Dataset({
    features: ['text', 'label'],
    num_rows: 960000
})

In [ ]:
show_random_elements(dataset["train"])

,text,label
0,"Positiv: nicht zu teuer, schöner Stoff, gut zu verschließen, gute Qualität Negativ: macht nicht was es soll Fazit: Es sollte meine Brüste (Für ein Kleid) etwas heben aber es gibt keinen Unterschied, die Brust sieht nur Unförmiger aus. Vielleicht ist dieser Effekt nur bis zu einer bestimmten Körbchengröße möglich... 2 Sterne da die Qualität gut ist",0
1,温風も出ると期待して購入しました。ドライヤーやファンヒーターのような温風を期待しましたが勢いや風量はもの足りません。ただのサーキュレーターとしては問題ないです。,0
2,buen producto buen relacion precio calidad.es muy mona y atractiva el diseño.aunque cuesta bastante trabajo moler a mano y cansa para hacer cantidad ..eléctrica sería mejor,1
3,La lampara llego rota. Además la bombilla que trae da destellos cuando esta apagada.,0
4,Super schnelle Lieferung,1
5,"Mir ist es gleich ergangen wie schon einer Person hier auf den Rezensionen, ich habe auch nicht gelesen, dass man die Brille nicht zurück schicken kann :-( das ist aber das einzige, was man mit diesem echt unglaublich hässlichen Teil machen kann. Die Gläser sind auch nicht rund wie auf der Abbildung sondern oval. Sehr schlechte Qualität und Gläser verkratzt. Finger weg!",0
6,"Ich habe auch schonmal ein Buch geschrieben, aber einfach so für mich. Klar, habe ich schonmal drüber nachgedacht wie es wäre wenn ich das Buch bei Amazon anbieten würde und noch mehr Leute mein Buch lesen würden. Dieses Buch erklärt einem Schritt für Schritt wie man es angehen muss. Und es ist wirklich gar nicht schwer, man muss nur wissen wie...",1
7,捏着鼻子耐心看完大段无聊的滑雪情节，满心期待最后的转折，结果就这个鬼！凑字数灌水也没这样的，这本超过了“美丽的凶器”成为我心目中大神最烂的作品。,0
8,So far so good. Opened box and picture looked clear when testing it out.,1
9,風量も十分でしたが、説明では4枚羽とのことでしたが届いた物は3枚羽でした。 あと充電中はLEDが点灯するとの説明ですがLEDが見当たりません。,1


In [ ]:
examples_per_class = 16
train_examples = {"text": [], "label": []}

In [ ]:
train_ds = dataset["train"].shuffle(seed=42)

In [ ]:
num_pos = 0
num_neg = 0
i = 0
while (num_pos != examples_per_class or num_neg != examples_per_class):
  if train_ds[i]['label'] == 1 and num_pos < 16:
    train_examples['text'].append(train_ds[i]['text'])
    train_examples['label'].append(1)
    num_pos += 1

  if train_ds[i]['label'] == 0 and num_neg < 16:
    train_examples['text'].append(train_ds[i]['text'])
    train_examples['label'].append(0)
    num_neg += 1

  i += 1

In [ ]:
from datasets import Dataset

In [ ]:
new_ds = Dataset.from_dict(train_examples)

In [ ]:
new_ds, new_ds[0]

(Dataset({
     features: ['text', 'label'],
     num_rows: 32
 }),
 {'text': 'I liked the convenience of ordering this online and that it arrived promptly.',
  'label': 1})

In [ ]:
test_ds = dataset["test"]

In [ ]:
test_ds

Dataset({
    features: ['text', 'label'],
    num_rows: 24000
})

In [ ]:
show_random_elements(new_ds, 16)

,text,label
0,按建议尺码表买的，大了。。。。而且我还按那个尺码买了五条不一样的裤子。。。。之前没有退货经验。。。难道要退回国外。。。,0
1,"Does not work, bought two and regret purchase. DO NOT BUY",0
2,"Es fantástica, la recomiendo para los fans de Harry Potter",1
3,This is a very solid wireless charger that has aluminum case and beautiful fabric surface. It’s super fast and great quality and working with my otterbox case. My phone is iphone xr.,1
4,This product advertised to be a set of 4 but you will only receive one bottle of shampoo. Save your money and buy elsewhere!!!,0
5,"I would buy again,I only used it in my super coffee so far",1
6,"calidad pésima y pequeño como el solo, no recomendable, material plástico y la parte metalica es muy endeble, para nada recomendable.",0
7,Muy grande para el reloj de la esfera pequeña... en mi opinión no debería de ser tan ancha si es la esfera de 38. También tendrían que hacer distinción entre femenino y masculino,0
8,I liked the convenience of ordering this online and that it arrived promptly.,1
9,"Livraison rapide et produit bien emballé. Néanmoins je ne recommande pas du tout car cette nappes n'est pas antidérapante, elle est même plus glissante qu'une nappe en tissu. Déception",0


In [ ]:
# Load SetFit model from Hub
ckpt = "distiluse-base-multilingual-cased-v2"
model = SetFitModel.from_pretrained(ckpt)

# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=new_ds,
    eval_dataset=test_ds,
    loss_class=CosineSimilarityLoss,
    batch_size=32,
    num_epochs=20,
)

config.json not found in HuggingFace Hub


Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/114 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/531 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/341 [00:00<?, ?B/s]

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [ ]:
# Train and evaluate!
trainer.train()

***** Running training *****
  Num examples = 1280
  Num epochs = 20
  Total optimization steps = 40
  Total train batch size = 32


Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Iteration:   0%|          | 0/40 [00:00<?, ?it/s]

Iteration:   0%|          | 0/40 [00:00<?, ?it/s]

Iteration:   0%|          | 0/40 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/40 [00:00<?, ?it/s]

Iteration:   0%|          | 0/40 [00:00<?, ?it/s]

Iteration:   0%|          | 0/40 [00:00<?, ?it/s]

Iteration:   0%|          | 0/40 [00:00<?, ?it/s]

Iteration:   0%|          | 0/40 [00:00<?, ?it/s]

In [ ]:
trainer.evaluate()

***** Running evaluation *****


{'accuracy': 0.7044583333333333}

In [ ]:
! git config --global user.email "user@provider.com"

In [ ]:
trainer.push_to_hub("setfit-distiluse-base-multilingual-cased-v2-finetuned-amazon-reviews-multi-binary")

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:42: FutureWarning: Deprecated positional argument(s) used in 'push_to_hub': pass repo_path_or_name='setfit-distiluse-base-multilingual-cased-v2-finetuned-amazon-reviews-multi-binary', repo_url=None, commit_message='Add SetFit model', organization=None, private=None, api_endpoint=None, use_auth_token=None, git_user=None, git_email=None, config=None, skip_lfs_files=False as keyword args. From version 0.12 passing these as positional arguments will result in an error,
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'push_to_hub': repo_path_or_name. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)
Cloning https://huggingface.co/mrm8488/setfit-distiluse-base-multilingual-cased-v2-finetuned-amazon-reviews-multi-binary into local empty directory.


Upload file pytorch_model.bin:   0%|          | 3.34k/514M [00:00<?, ?B/s]

Upload file model_head.pkl:  67%|######7   | 3.34k/4.95k [00:00<?, ?B/s]

Upload file 2_Dense/pytorch_model.bin:   0%|          | 3.34k/1.50M [00:00<?, ?B/s]